In [21]:
import requests
from bs4 import BeautifulSoup
import re
import urllib.request
import json
import os
from datetime import datetime

In [22]:
url_AENA_IATA_aeropuertos = "https://raw.githubusercontent.com/abediaga/AENA/main/AENA_IATA_aeropuertos.json"

with urllib.request.urlopen(url_AENA_IATA_aeropuertos) as url:
    aeropuertos = json.loads(url.read().decode())

In [23]:
url = "http://www.aena.es/es/aerolineas/red-aeropuertos.html"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

contenido = soup.find_all("div", {"class": "datos_interes"})

has_changed = False

if len(contenido) == 2 :
    div_aeropuertos = contenido[1]
    
    a_aeropuertos = div_aeropuertos.findAll("a")
    
    for a_aeropuerto in a_aeropuertos:
        #print(a_aeropuerto.text.strip().partition("("))
        #print(a_aeropuerto.text[a_aeropuerto.text.find("(")+1:a_aeropuerto.text.find(")")])
        sep = '\((.*?)\)'
        nombre = re.split(sep,a_aeropuerto.text,1)[0].strip().upper()
        codigo_IATA = re.split(sep,a_aeropuerto.text,1)[1].strip()
        
        if codigo_IATA in aeropuertos:
            otros_nombres = []
            if "Otros" in aeropuertos[codigo_IATA]:
                otros_nombres = aeropuertos[codigo_IATA]["Otros"]
            if (nombre != aeropuertos[codigo_IATA]["Nombre"]) and (nombre not in otros_nombres):
                print("Nombre del aeropuerto nuevo <" + str(nombre) + "> sobre un código IATA existente.")
                #Añadimos el nombre del aeropuerto como nombre principal y pasamos el nombre principal antiguo como otro nombre
                if "Otros" in aeropuertos[codigo_IATA]:
                    aeropuertos[codigo_IATA]["Otros"].append(aeropuertos[codigo_IATA]["Nombre"])
                else:
                    aeropuertos[codigo_IATA]["Otros"] = []
                    aeropuertos[codigo_IATA]["Otros"].append(aeropuertos[codigo_IATA]["Nombre"])
                aeropuertos[codigo_IATA]["Nombre"] = nombre
                has_changed = True
        else:
            #Añadimos el aeropuerto a la VARIABLE aeropuertos
            print("El aeropuerto " + str(nombre) + " no existe. Se añade a la base de datos.")
            aeropuertos[codigo_IATA] = {}
            aeropuertos[codigo_IATA]["Nombre"] = nombre
            has_changed = True

In [25]:
if has_changed:
    print("Actualizar el fichero JSON")
    today = datetime.today().strftime('%Y-%m-%d')
    os.rename(r'AENA_IATA_aeropuertos.json',r'AENA_IATA_aeropuertos_" + str(today) + ".json')
    with open('AENA_IATA_aeropuertos.json', 'w') as file:
        json.dump(aeropuertos, file)
else:
    print("No hay que actualizar el fichero JSON")

No hay que actualizar el fichero JSON
